In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.utils import shuffle
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from geopy.geocoders import Nominatim
import folium
import random

In [2]:
random.seed(123)
np.random.seed(123)
tf.random.set_seed(123)

In [3]:
orders = pd.read_csv('data/olist_orders_dataset.csv')
order_items = pd.read_csv('data/olist_order_items_dataset.csv')
order_payments = pd.read_csv('data/olist_order_payments_dataset.csv')
order_reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')
products = pd.read_csv('data/olist_products_dataset.csv')
sellers = pd.read_csv('data/olist_sellers_dataset.csv')
customers = pd.read_csv('data/olist_customers_dataset.csv')
geolocation = pd.read_csv('data/olist_geolocation_dataset.csv')
products_translation = pd.read_csv('data/product_category_name_translation.csv')

# 1. Data Exploration and Analysis


## 1.1. Dropping columns that would not influence the models

## - orders Exploration

In [4]:
orders_new = orders.drop(['order_approved_at', 'order_purchase_timestamp'], axis=1)

## - order_items exploration

In [5]:
order_items_new = order_items.drop(['seller_id', 'shipping_limit_date', 'freight_value'], axis = 1)

## - order_payments exploration

In [6]:
order_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


###### Drop this dataset because the information is not required for the models

## - order_reviews exploration

In [7]:
order_reviews_new = order_reviews.drop(['review_comment_title' , 'review_comment_message' , 'review_creation_date' , 'review_answer_timestamp'], axis = 1)

## - products exploration

In [8]:
products_new = products.drop(['product_name_lenght', 'product_description_lenght', 'product_weight_g'], axis = 1)

In [9]:
# Merge the products DataFrame with the products_translation DataFrame
products_merged = products_new.merge(products_translation, 
                                 left_on='product_category_name', 
                                 right_on='product_category_name', 
                                 how='left')

# Drop the original 'product_category_name' column
products_merged.drop('product_category_name', axis=1, inplace=True)

# Optionally, rename the 'product_category_name_english' column to 'product_category_name'
products_merged.rename(columns={'product_category_name_english': 'product_category_name'}, inplace=True)

## - sellers exploration

In [10]:
sellers_new = sellers.drop(['seller_zip_code_prefix'], axis = 1)

## - customers exploration

In [11]:
customers_new = customers.drop(['customer_unique_id'], axis=1)

## - geolocation exploration

In [12]:
geolocation = geolocation.groupby(['geolocation_city','geolocation_state','geolocation_zip_code_prefix']).last()

# 1.2. Merging of datasets 

In [13]:
# Merge datasets related to orders
orders_forecasting = orders \
    .merge(order_items_new, on='order_id', how='left') \
    .merge(order_reviews_new, on='order_id', how='left') \
    .merge(products_merged, on='product_id', how='left') \
    .merge(customers_new, on='customer_id', how='left')

In [14]:
orders_forecasting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114092 entries, 0 to 114091
Data columns (total 21 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       114092 non-null  object 
 1   customer_id                    114092 non-null  object 
 2   order_status                   114092 non-null  object 
 3   order_purchase_timestamp       114092 non-null  object 
 4   order_approved_at              113930 non-null  object 
 5   order_delivered_carrier_date   112112 non-null  object 
 6   order_delivered_customer_date  110839 non-null  object 
 7   order_estimated_delivery_date  114092 non-null  object 
 8   order_item_id                  113314 non-null  float64
 9   product_id                     113314 non-null  object 
 10  price                          113314 non-null  float64
 11  review_id                      113131 non-null  object 
 12  review_score                  

In [15]:
orders_forecasting.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,review_id,review_score,product_photos_qty,product_length_cm,product_height_cm,product_width_cm,product_category_name,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,...,a54f0611adc9ed256b57ede6b6eb5114,4.0,4.0,19.0,8.0,13.0,housewares,3149,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,595fac2a385ac33a80bd5114aec74eb8,...,8d5266042046a06655c8db133d120ba5,4.0,1.0,19.0,13.0,19.0,perfumery,47813,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,aa4383b373c6aca5d8797843e5594415,...,e73b67b67587f7644d5bd1a52deb1b01,5.0,1.0,24.0,19.0,21.0,auto,75265,vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,...,359d03e676b3c069f62cadba8dd3f6e8,5.0,3.0,30.0,10.0,20.0,pet_shop,59296,sao goncalo do amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1.0,65266b2da20d04dbe00c5c2d3bb7859e,...,e50934924e227544ba8246aeb3770dd4,5.0,4.0,51.0,15.0,15.0,stationery,9195,santo andre,SP


In [16]:
full_order_details = pd.merge(geolocation, orders_forecasting, left_on='geolocation_zip_code_prefix', right_on='customer_zip_code_prefix', how='right')
route_optimisation = full_order_details[['customer_zip_code_prefix','geolocation_lat','geolocation_lng','order_delivered_carrier_date','order_id','customer_city']]
route_optimisation.drop_duplicates(inplace=True)
route_optimisation['order_delivered_carrier_date'] = route_optimisation['order_delivered_carrier_date'].str[:10]
route_optimisation.head(5)

C:\Users\Mindy\AppData\Local\Temp\ipykernel_3900\3540856820.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_optimisation.drop_duplicates(inplace=True)
C:\Users\Mindy\AppData\Local\Temp\ipykernel_3900\3540856820.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_optimisation['order_delivered_carrier_date'] = route_optimisation['order_delivered_carrier_date'].str[:10]


,customer_zip_code_prefix,geolocation_lat,geolocation_lng,order_delivered_carrier_date,order_id,customer_city
0,3149,-23.575377,-46.587410,2017-10-04,e481f51cbdc54678b7cc49136f2d6af7,sao paulo
1,3149,-23.583452,-46.586284,2017-10-04,e481f51cbdc54678b7cc49136f2d6af7,sao paulo
2,47813,-12.124719,-45.011148,2018-07-26,53cdb2fc8bc7dce0b6741e2150273451,barreiras
3,75265,-16.743570,-48.511633,2018-08-08,47770eb9100c2d0c44946d9cf07ec65d,vianopolis
4,75265,-16.746170,-48.522116,2018-08-08,47770eb9100c2d0c44946d9cf07ec65d,vianopolis


# 2. Business Problem

## 2.1. Random Forest

In [17]:
df = orders_forecasting

In [18]:
# Preprocess datetime columns: Convert to pandas datetime type
datetime_features = ['order_purchase_timestamp', 'order_approved_at',
                     'order_delivered_carrier_date', 'order_delivered_customer_date',
                     'order_estimated_delivery_date']
for feature in datetime_features:
    df[feature] = pd.to_datetime(df[feature])

# Drop rows with NaN values in the target variable
df = df.dropna(subset=['review_score'])

# Assuming 'review_score' is the target and the rest are features
X = df.drop('review_score', axis=1)
y = df['review_score']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Preprocessing pipelines for both numerical and categorical data
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_features.remove('product_photos_qty')  # Assuming this is a count and does not need scaling
categorical_features = ['order_status', 'product_category_name']  # Assuming these are the categorical features

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputation with mean for numeric columns
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))  # One-hot encode and ensure dense output
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Preprocess the training and test data
X_train_prepared = preprocessor.fit_transform(X_train)
X_test_prepared = preprocessor.transform(X_test)

# Create and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_prepared, y_train)

# Predict and evaluate the Random Forest model
rf_pred = rf_model.predict(X_test_prepared)
rf_mse = mean_squared_error(y_test, rf_pred)
print(f"Random Forest MSE: {rf_mse}")

C:\Users\Mindy\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Random Forest MSE: 1.7931732579606117


In [19]:
# Get feature importances from the model
feature_importances = rf_model.feature_importances_

# Assuming preprocessor is your ColumnTransformer with pipelines for 'num' and 'cat' preprocessing
numeric_feature_names = numeric_features  # Assuming this variable is defined with your numeric feature names

# Adjust retrieval of categorical feature names after one-hot encoding to accommodate your pipeline's structure
# Note: Ensure 'cat' matches the name given in your ColumnTransformer for the categorical handling step
categorical_feature_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out()

# Combine both lists of feature names: numerics remain unchanged, categoricals are expanded post one-hot encoding
all_feature_names = numeric_feature_names + list(categorical_feature_names)

# Create a DataFrame to display feature importances
feature_importance_df = pd.DataFrame({
    'feature': all_feature_names,
    'importance': feature_importances
}).sort_values(by='importance', ascending=False)

# Display the top 10 features ranked by importance
print(feature_importance_df.head(10))

                     feature  importance
5   customer_zip_code_prefix    0.300528
1                      price    0.184720
3          product_height_cm    0.096482
2          product_length_cm    0.088660
4           product_width_cm    0.084864
9               x0_delivered    0.075412
0              order_item_id    0.025145
80         x1_sports_leisure    0.008672
57          x1_health_beauty    0.008069
63             x1_housewares    0.007619


In [20]:
# Random Forest predictions
rf_pred = rf_model.predict(X_test_prepared)

# Calculate R-squared for Random Forest
rf_r2 = r2_score(y_test, rf_pred)
print(f"Random Forest R-squared: {rf_r2}")

# Calculate RMSE for Random Forest
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
print(f"Random Forest RMSE: {rf_rmse}")

# Calculate MAE for Random Forest
rf_mae = mean_absolute_error(y_test, rf_pred)
print(f"Random Forest MAE: {rf_mae}")

Random Forest R-squared: 0.07353828386788552
Random Forest RMSE: 1.339094193087481
Random Forest MAE: 1.0374572056197657


## 2.2. Neural Network

In [21]:
# Define a Neural Network model
nn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_prepared.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

# Compile the Neural Network model
nn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the Neural Network model
nn_model.fit(X_train_prepared, y_train, epochs=10, batch_size=32, verbose=1)

# Evaluate the Neural Network model using the test data
nn_mse = nn_model.evaluate(X_test_prepared, y_test, verbose=0)
print(f"Neural Network MSE: {nn_mse}")

Epoch 1/10
2829/2829 [==============================] - 16s 5ms/step - loss: 1.8879
Epoch 2/10
2829/2829 [==============================] - 12s 4ms/step - loss: 1.7794
Epoch 3/10
2829/2829 [==============================] - 11s 4ms/step - loss: 1.7684
Epoch 4/10
2829/2829 [==============================] - 12s 4ms/step - loss: 1.7632
Epoch 5/10
2829/2829 [==============================] - 11s 4ms/step - loss: 1.7567
Epoch 6/10
2829/2829 [==============================] - 11s 4ms/step - loss: 1.7507
Epoch 7/10
2829/2829 [==============================] - 14s 5ms/step - loss: 1.7453
Epoch 8/10
2829/2829 [==============================] - 13s 5ms/step - loss: 1.7396
Epoch 9/10
2829/2829 [==============================] - 13s 5ms/step - loss: 1.7353
Epoch 10/10
2829/2829 [==============================] - 12s 4ms/step - loss: 1.7328
Neural Network MSE: 1.7322440147399902


In [22]:
# Neural Network predictions
nn_pred = nn_model.predict(X_test_prepared).flatten() # Flatten if necessary

# Calculate R-squared for Neural Network
nn_r2 = r2_score(y_test, nn_pred)
print(f"Neural Network R-squared: {nn_r2}")

# Calculate RMSE for Neural Network
nn_rmse = np.sqrt(mean_squared_error(y_test, nn_pred))
print(f"Neural Network RMSE: {nn_rmse}")

# Calculate MAE for Neural Network
nn_mae = mean_absolute_error(y_test, nn_pred)
print(f"Neural Network MAE: {nn_mae}")

708/708 [==============================] - 3s 3ms/step
Neural Network R-squared: 0.10501800789124849
Neural Network RMSE: 1.3161474186574504
Neural Network MAE: 1.0371709433948335


# 3. Demand Forecasting

## 3.1. Random Forest

In [23]:
# Convert datetime columns to pandas datetime type
datetime_features = ['order_purchase_timestamp', 'order_approved_at',
                     'order_delivered_carrier_date', 'order_delivered_customer_date',
                     'order_estimated_delivery_date']
for feature in datetime_features:
    df[feature] = pd.to_datetime(df[feature])

# Drop rows with NaN values in crucial columns for analysis
df = df.dropna(subset=['product_category_name', 'price'])

# Aggregate to calculate total orders per product_category_name
df_agg = df.groupby('product_category_name').size().reset_index(name='total_orders')

# Merge aggregated total orders back to the original dataframe
df = df.merge(df_agg, on='product_category_name', how='left')

# Prepare features (X) and target (y)
X = df.drop(['total_orders', 'order_item_id'], axis=1)  # Adjust if needed
y = df['total_orders']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Define preprocessing pipelines for numeric and categorical data
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = ['order_status', 'product_category_name']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Preprocess the training and test data
X_train_prepared = preprocessor.fit_transform(X_train)
X_test_prepared = preprocessor.transform(X_test)

# Train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=123)
rf_model.fit(X_train_prepared, y_train)

# Predict on the test set and calculate evaluation metrics
y_pred = rf_model.predict(X_test_prepared)
rf_r2 = r2_score(y_test, y_pred)
rf_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rf_mae = mean_absolute_error(y_test, y_pred)

# Extract and display feature importances
importances = rf_model.feature_importances_
# Handling feature names for both numeric and one-hot encoded categorical features
try:
    # For scikit-learn 0.22 and newer
    categorical_feature_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out()
except AttributeError:
    # Fallback for older versions of scikit-learn
    categorical_feature_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names(categorical_features)
all_feature_names = numeric_features + list(categorical_feature_names)
features_df = pd.DataFrame({
    'Feature': all_feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Create a DataFrame to compare actual and predicted numbers of orders
predictions_df = pd.DataFrame({
    'Actual Orders': y_test.reset_index(drop=True),  # Reset index for alignment
    'Predicted Orders': y_pred
})

# Display the feature importances and the first 10 entries of actual vs predicted orders
print("Feature Importances:")
print(features_df.head(10))

print("\nPredicted vs Actual Orders:")
print(predictions_df.head(10))

# Print model evaluation metrics
print(f"\nRandom Forest R-squared: {rf_r2}")
print(f"Random Forest RMSE: {rf_rmse}")
print(f"Random Forest MAE: {rf_mae}")

C:\Users\Mindy\AppData\Local\Temp\ipykernel_3900\2508384973.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature] = pd.to_datetime(df[feature])
C:\Users\Mindy\AppData\Local\Temp\ipykernel_3900\2508384973.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature] = pd.to_datetime(df[feature])
C:\Users\Mindy\AppData\Local\Temp\ipykernel_3900\2508384973.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

Feature Importances:
                     Feature  Importance
21         x1_bed_bath_table    0.278054
57          x1_health_beauty    0.165164
53        x1_furniture_decor    0.116787
79         x1_sports_leisure    0.114151
29  x1_computers_accessories    0.108290
63             x1_housewares    0.075628
84          x1_watches_gifts    0.045261
83                   x1_toys    0.014029
82              x1_telephony    0.013064
19                   x1_auto    0.012983

Predicted vs Actual Orders:
   Actual Orders  Predicted Orders
0           2039            2039.0
1           6943            6943.0
2           4213            4213.0
3           2507            2507.0
4           9645            9645.0
5           8331            8331.0
6           4091            4091.0
7          11137           11137.0
8           6943            6943.0
9           5950            5950.0

Random Forest R-squared: 0.99999999880877
Random Forest RMSE: 0.11578077283779242
Random Forest MAE: 0.0020076749

## 3.2. Neural Network

In [ ]:
# Neural Network Model Training
nn_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_prepared.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

nn_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

nn_history = nn_model.fit(X_train_prepared, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Predict on the test set with the Neural Network
y_pred_nn = nn_model.predict(X_test_prepared).flatten()

# Evaluation metrics for the Neural Network
nn_r2 = r2_score(y_test, y_pred_nn)
nn_rmse = np.sqrt(mean_squared_error(y_test, y_pred_nn))
nn_mae = mean_absolute_error(y_test, y_pred_nn)

# Function to calculate permutation feature importance
def calculate_permutation_importance(model, X_test_prepared, y_test, feature_names):
    baseline_mse = mean_squared_error(y_test, model.predict(X_test_prepared))
    importance_scores = []

    for i, name in enumerate(feature_names):
        X_test_shuffled = X_test_prepared.copy()
        X_test_shuffled[:, i] = shuffle(X_test_shuffled[:, i])

        shuffled_mse = mean_squared_error(y_test, model.predict(X_test_shuffled))
        importance = shuffled_mse - baseline_mse
        importance_scores.append((name, importance))

    importance_scores.sort(key=lambda x: x[1], reverse=True)
    return importance_scores

# Get all feature names from the preprocessing pipeline
numeric_feature_names = numeric_features
categories = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features)
categorical_feature_names = [f"{cat}" for cat in categories]
all_feature_names = numeric_feature_names + list(categorical_feature_names)

# Create and display a DataFrame for Neural Network predictions vs actual orders
nn_predictions_df = pd.DataFrame({
    'Actual Orders': y_test.reset_index(drop=True),
    'Predicted Orders': y_pred_nn
})

# Calculate permutation feature importance for the Neural Network
nn_feature_importance = calculate_permutation_importance(nn_model, X_test_prepared, y_test, all_feature_names)

# Display the top 10 most important features
print("Top 10 Feature Importances from Neural Network:")
for feature, importance in nn_feature_importance[:10]:
    print(f"{feature}: {importance}")

print("\nNeural Network Predicted vs Actual Orders:")
print(nn_predictions_df.head(10))

print(f"Neural Network R-squared: {nn_r2}")
print(f"Neural Network RMSE: {nn_rmse}")
print(f"Neural Network MAE: {nn_mae}")

Epoch 1/100
2215/2215 [==============================] - 13s 5ms/step - loss: 12716122.0000 - val_loss: 846169.5000
Epoch 2/100
2215/2215 [==============================] - 10s 5ms/step - loss: 239397.7031 - val_loss: 44829.6172
Epoch 3/100
2215/2215 [==============================] - 9s 4ms/step - loss: 20763.8359 - val_loss: 7623.2676
Epoch 4/100
2215/2215 [==============================] - 10s 4ms/step - loss: 4317.0620 - val_loss: 1873.8275
Epoch 5/100
2215/2215 [==============================] - 10s 4ms/step - loss: 1095.0189 - val_loss: 684.6666
Epoch 6/100
2215/2215 [==============================] - 10s 4ms/step - loss: 338.9482 - val_loss: 294.5088
Epoch 7/100
2215/2215 [==============================] - 11s 5ms/step - loss: 138.1505 - val_loss: 196.7446
Epoch 8/100
2215/2215 [==============================] - 12s 5ms/step - loss: 84.2305 - val_loss: 149.7566
Epoch 9/100
2215/2215 [==============================] - 10s 5ms/step - loss: 63.5699 - val_loss: 119.7059
Epoch 10/100

## 3.3. XGBoost

In [ ]:
# Convert the prepared training and test datasets into DMatrix objects, optimized for XGBoost
dtrain = xgb.DMatrix(X_train_prepared, label=y_train)
dtest = xgb.DMatrix(X_test_prepared, label=y_test)

# Define XGBoost model parameters
params = {
    'max_depth': 6,
    'eta': 0.3,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

# Specify the number of boosting rounds
num_boost_round = 100

# Train the XGBoost model
bst = xgb.train(params, dtrain, num_boost_round)

# Predict on the test set with XGBoost
y_pred_xgb = bst.predict(dtest)

# Evaluation metrics for the XGBoost model
xgb_r2 = r2_score(y_test, y_pred_xgb)
xgb_rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
xgb_mae = mean_absolute_error(y_test, y_pred_xgb)

# Get feature importance and display
importance_weight = bst.get_score(importance_type='weight')
importance_gain = bst.get_score(importance_type='gain')

importance_weight_df = pd.DataFrame(list(importance_weight.items()), columns=['Feature', 'Score']).sort_values(by='Score', ascending=False)
importance_gain_df = pd.DataFrame(list(importance_gain.items()), columns=['Feature', 'Score']).sort_values(by='Score', ascending=False)

print("\nXGBoost Feature Importance based on weight:")
print(importance_weight_df.head(10))

print("\nXGBoost Feature Importance based on gain:")
print(importance_gain_df.head(10))

# Print evaluation metrics
print(f"\nXGBoost R-squared: {xgb_r2}")
print(f"XGBoost RMSE: {xgb_rmse}")
print(f"XGBoost MAE: {xgb_mae}")

# Optionally, create and display a DataFrame for XGBoost predictions vs actual orders
xgb_predictions_df = pd.DataFrame({
    'Actual Orders': y_test.reset_index(drop=True),  # Ensure alignment between actual and predicted
    'Predicted Orders': y_pred_xgb
})

print("\nXGBoost Predicted vs Actual Orders:")
print(xgb_predictions_df.head(10))

# 4. Route Optimisation

## 4.1. Clustering by parcel size

In [ ]:
df['Product_volume'] = df['product_length_cm']* df['product_height_cm']* df['product_width_cm']
df.dropna(subset=['Product_volume'], inplace=True)
print(df.head())

In [ ]:
# Number of parcels (desiered number of clusters)
# Super small, small, meduim, big, Super big
max_num_size=5

In [ ]:
# Cluster the parcels based on the size of the parcel
kmeans = KMeans(n_clusters=max_num_size)
# Reshape the input data to a 2D array
kmeans.fit(df['Product_volume'].values.reshape(-1, 1))
cluster_centers = kmeans.cluster_centers_
cluster_labels = kmeans.labels_

# Visualizing the clusters
plt.figure(figsize=(8, 6))
plt.scatter(df['Product_volume'], np.zeros_like(df['Product_volume']), c=cluster_labels, cmap='viridis', s=50)
plt.scatter(cluster_centers, np.zeros_like(cluster_centers), c='red', marker='X', s=300, label='Cluster Centers')
plt.title('Clustering of Parcels based on Volumes')
plt.xlabel('Volume')
plt.legend()
plt.grid(True)
plt.show()

# Printing cluster centers
print("Cluster Centers:")
for i, center in enumerate(cluster_centers):
    print(f"Cluster {i+1}: {center[0]}")

In [ ]:
# Cluster Centers (derived from result above)
cluster_centers = {
  'Cluster 1': 22953.805372673138,
  'Cluster 2': 102682.53763440774,
  'Cluster 3': 53206.30492355889,
  'Cluster 4': 4724.973529788527,
  'Cluster 5': 211177.80929095377
}

# Define categories based on cluster centers (tag the parcels with different sizes)
categories = {
    'super small': cluster_centers['Cluster 4'],
    'small': cluster_centers['Cluster 1'],
    'medium': cluster_centers['Cluster 3'],
    'big': cluster_centers['Cluster 2'],
    'super big': cluster_centers['Cluster 5']
}

# Assign parcels to categories based on cluster centers
def assign_parcels_to_categories(cluster_centers, categories, product_volumes):
    assigned_categories = []
    for volume in product_volumes:
        for category, threshold in categories.items():
            if volume <= threshold:
                assigned_categories.append((volume, category))
                break
    return assigned_categories

# Assign parcels to categories
assigned_categories = assign_parcels_to_categories(cluster_centers, categories, df['Product_volume'])

# Print the assigned categories for each parcel volume
# print("Assigned categories for each parcel volume:")
# for volume, category in assigned_categories:
#    print(f"Volume: {volume}, Category: {category}")

# A dictionary to store the count of parcels in each category
category_counts = {category: 0 for category in categories.keys()}

# Count the number of parcels in each category
for _, category in assigned_categories:
    category_counts[category] += 1

# Print the number of parcels in each category
print("Number of parcels in each category:")
for category, count in category_counts.items():
    print(f"{category}: {count}")

In [ ]:
# number of parcels in each cluster (derived from result above)
parcel_counts = {
    'super small': 46606,
    'small': 44636,
    'medium': 15251,
    'big': 5374,
    "super big": 1252
}

# van capacity (maximum volume of parcels that van can load)
van_capacity = 500000000

In [ ]:
# Initialize variables
loaded_parcels = {}
remaining_capacity = van_capacity

# Iterate through categories and allocate parcels proportionally
for category, count in parcel_counts.items():
    # Calculate the proportion of the van capacity for the current category
    category_proportion = count / sum(parcel_counts.values())
    # Calculate the volume of parcels to be loaded for this category
    category_volume = van_capacity * category_proportion
    # Update the loaded parcels with the rounded count
    loaded_parcels[category] = round(category_volume / categories[category])
    # Update the remaining capacity
    remaining_capacity -= loaded_parcels[category] * categories[category]

# Print the loaded parcels
print("Optimal combination of parcels:")
for category, count in loaded_parcels.items():
    print(f"{count} {category}")

## 4.2. Clustering by parcel size

### 4.2.1. Data filtering & cleaning

In [ ]:
# Filter the dataset for rows where order_delivered_carrier_date is '2017-05-05' and customer_city in 'sao paulo'
route_optimisation = route_optimisation.dropna(subset=['order_delivered_carrier_date'])
filtered_data = route_optimisation[
    (route_optimisation['order_delivered_carrier_date'].str.startswith('2017-05-05')) &
    (route_optimisation['customer_city'] == "sao paulo")]
print(filtered_data)
print(filtered_data.count())

# Extract latitude and longitude coordinates from filtered_data
latitudes = filtered_data['geolocation_lat']
longitudes = filtered_data['geolocation_lng']

In [ ]:
# Plot the geolocations
plt.figure(figsize=(8, 6))
plt.scatter(filtered_data['geolocation_lng'], filtered_data['geolocation_lat'], color='blue', label='Geolocations')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Geolocations with order_delivered_carrier_date_y = 2017-05-05')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Create a map centered at the mean latitude and longitude of the filtered data
map = folium.Map(location=[filtered_data['geolocation_lat'].mean(), filtered_data['geolocation_lng'].mean()], zoom_start=10)

# Add markers for each geolocation point
for idx, row in filtered_data.iterrows():
    folium.Marker([row['geolocation_lat'], row['geolocation_lng']]).add_to(map)

# Display the map
map

## 4.2.2. Modelling 

In [ ]:
# Number of drivers (desired number of clusters)
max_num_drivers = 5

In [ ]:
# Combine latitudes and longitudes into a 2D array
locations = np.column_stack((latitudes, longitudes))

# Cluster the locations based on the number of drivers
kmeans = KMeans(n_clusters=max_num_drivers)
kmeans.fit(locations)
cluster_centers = kmeans.cluster_centers_
labels = kmeans.labels_

# Visualize the clusters
plt.figure(figsize=(8, 6))

for i in range(max_num_drivers):
    # Plot points belonging to cluster i
    plt.scatter(locations[labels == i, 1], locations[labels == i, 0], label=f'Cluster {i+1}')

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Clustered Geolocations')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Create a map centered at the mean latitude and longitude of the cluster centers
clustered_map = folium.Map(location=[cluster_centers[:, 0].mean(), cluster_centers[:, 1].mean()], zoom_start=10)

# Generate random colors for each cluster
colors = ['#' + ''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(max_num_drivers)]

# Add markers for each point, color-coded by cluster
for i in range(max_num_drivers):
    cluster_points = locations[labels == i]
    for point in cluster_points:
        folium.CircleMarker(location=[point[0], point[1]], radius=5, color=colors[i], fill=True, fill_color=colors[i], fill_opacity=0.7).add_to(clustered_map)

# Display the map
clustered_map

## 4.2.3. Evaluation of clustering model

In [ ]:
np.random.seed(123)

# Range of clusters to evaluate
min_clusters = 1
max_clusters = max_num_drivers

# Evaluation for each cluster
avg_distances = []
avg_num_points = []
avg_total_distances = []

for num_clusters in range(min_clusters, max_clusters + 1):
    # Step 2: Cluster the locations based on the number of clusters
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(locations)
    cluster_centers = kmeans.cluster_centers_
    labels = kmeans.labels_
    
    # Calculate average distance for each cluster
    avg_distance = 0
    for i in range(num_clusters):
        distances = np.linalg.norm(locations[labels == i] - cluster_centers[i], axis=1)
        avg_distance += np.mean(distances)
    
    avg_distances.append(avg_distance / num_clusters)  # Divide by num_clusters to get average

    # Calculate average number of points for each cluster
    avg_num_points.append(len(locations) / num_clusters)

    # Calculate average total distance for each cluster
    total_distances = np.zeros(num_clusters)
    for i in range(num_clusters):
        distances = np.linalg.norm(locations[labels == i] - cluster_centers[i], axis=1)
        total_distances[i] = np.sum(distances)
    avg_total_distances.append(np.mean(total_distances))

In [ ]:
# Visualize average distances for each number of clusters
plt.plot(range(min_clusters, max_clusters + 1), avg_distances, marker='o', color="green")
plt.xlabel('Number of Clusters')
plt.ylabel('Average Distance')
plt.title('Evaluation of Clustering')
plt.grid(True)
plt.show()

In [ ]:
plt.plot(range(min_clusters, max_clusters + 1), avg_num_points, marker='o', color='blue')
plt.xlabel('Number of Clusters')
plt.ylabel('Average Number of Points')
plt.title('Average Number of Points in Each Cluster')
plt.grid(True)
plt.show()

In [ ]:
plt.plot(range(min_clusters, max_clusters + 1), avg_total_distances, marker='x', color='orange')
plt.xlabel('Number of Clusters')
plt.ylabel('Average Total Distance')
plt.title('Average Total Distance in Each Cluster')
plt.grid(True)
plt.show()

## 4.2.4. Optimal number of clusters based on evaluation

In [ ]:
# Optimal number of drivers (desired number of clusters)
optimal_drivers = 2

In [ ]:
# Create a map centered at the mean latitude and longitude of the cluster centers
optimal_map = folium.Map(location=[cluster_centers[:, 0].mean(), cluster_centers[:, 1].mean()], zoom_start=10)

# Generate random colors for each cluster
colors = ['#' + ''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(optimal_drivers)]

# Add markers for each point, color-coded by cluster
for i in range(optimal_drivers):
    cluster_points = locations[labels == i]
    for point in cluster_points:
        folium.CircleMarker(location=[point[0], point[1]], radius=5, color=colors[i], fill=True, fill_color=colors[i], fill_opacity=0.7).add_to(optimal_map)

# Display the map
optimal_map